In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import pandas as pd
from gensim.models import word2vec
import nltk
nltk.download('wordnet')
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet
from nltk.tokenize import word_tokenize 
%pip install contractions
import contractions
import string
from gensim.parsing.preprocessing import STOPWORDS
from collections import Counter
%pip install imbalanced-learn
from imblearn.under_sampling import RandomUnderSampler

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
     |████████████████████████████████| 321 kB 3.8 MB/s 
     |████████████████████████████████| 284 kB 40.5 MB/s 
  Created wheel for pyahocorasick: filename=pyahocorasick-1.4.2-cp37-cp37m-linux_x86_64.whl size=85439 sha256=e8bf84681b675f1ecdb53cfa41be3d5c68565cc532bd6f6383bd0024993f9a27
  Stored in directory: /root/.cache/pip/wheels/25/19/a6/8f363d9939162782bb8439d886469756271abc01f76fbd790f
Successfully built pyahocorasick


Importing a combined version of the dataset with 10k rows from each .tsv

In [3]:
df = pd.read_csv('/content/drive/MyDrive/ALDA_Project/amazon_reviews_us_combined.tsv',sep='\t',error_bad_lines=False )

Removing all reviews with a neutral rating


In [4]:
df = df[df.star_rating != 3]
df.shape

(334435, 15)

In [5]:
df.loc[df['star_rating'] > 3, 'star_rating_sentiment'] = 'positive'

df['star_rating_sentiment'] = df['star_rating_sentiment'].fillna('negative')


/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:1596: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = _infer_fill_value(value)
/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:1763: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(loc, value)
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the 

In [6]:
reviews = df["review_body"].astype('str').values
labels = df["star_rating_sentiment"].values

In [7]:
rus = RandomUnderSampler()
reviews = reviews.reshape(-1, 1)
reviews, labels = rus.fit_resample(reviews, labels)

In [14]:
len(reviews)

99332

In [8]:
all_stopwords = STOPWORDS.union(['it','at','me','br','if','to','do'])
lemmatizer = WordNetLemmatizer()

def get_wordnet_pos(word):

    tag = word[0].upper()
    tag_dict = {"J": wordnet.ADJ,
                "N": wordnet.NOUN,
                "V": wordnet.VERB,
                "R": wordnet.ADV}

    return tag_dict.get(tag, wordnet.NOUN)

def process_review(sentence):

  # convert the sentence to lower case, fix contrations and the otkenize
  sentence = word_tokenize(contractions.fix(sentence.lower()))

  # Removing all short words, punctuations and digits
  sentence = [word for word in sentence if len(word)>1 and all(punc not in word for punc in string.punctuation) and not any(char.isdigit() for char in word)]

  # remove all stop words
  sentence = [word for word in sentence if not word in all_stopwords]

  # add a parts of speech tag to each word
  sentence = nltk.pos_tag(sentence)

  # lemmatize each word using itself and its POS
  sentence = [lemmatizer.lemmatize(word[0], get_wordnet_pos(word[1])) for word in sentence]

  return sentence

In [9]:
processed_reviews = []
for review in reviews:
  processed_reviews.append(process_review(review[0]))


In [10]:
word_count = Counter()
sentences_in_words = []
for i in range(len(processed_reviews)):
    sentences_in_words.append(processed_reviews[i])
    
    for word in sentences_in_words[i]:
        
        if len(word)>1 and type(word)==str and string.punctuation not in word:
          word_count.update({str(word): 1})

In [11]:
model = word2vec.Word2Vec(sentences_in_words)

In [12]:
for word_pair in [['good','bad'],['best','worst'],['positive','negative'],['awesome','terrible']]:
  result = []
  correct_count = 0
  for i in range(len(processed_reviews)):
    sum_g = 0
    sum_b = 0
    n=0
    for word in processed_reviews[i]:
      try:
        sum_g += model.wv.similarity(word, word_pair[0])
        sum_b += model.wv.similarity(word, word_pair[1])
      except:
        n-=1
    if sum_g>0:
      avg_g = sum_g/(len(processed_reviews[i])+n)
    else:
      avg_g =0
    if sum_b>0:
      avg_b = sum_b/(len(processed_reviews[i])+n)
    else:
      avg_b =0
    if (avg_g>avg_b and labels[i]=='positive') or (avg_g<=avg_b and labels[i]=='negative'):
      correct_count+=1
    # print(labels[i],avg_g,avg_b,avg_g>avg_b)
  print(word_pair,correct_count/len(processed_reviews))

['good', 'bad'] 0.6631397736882374
['best', 'worst'] 0.5676720492892522
['positive', 'negative'] 0.5419099585229332
['awesome', 'terrible'] 0.6695828131921234


In [13]:
for word_pair in [['good','bad'],['best','worst'],['positive','negative'],['awesome','terrible']]:
  result = []
  correct_count = 0
  for i in range(len(processed_reviews)):
    sum=0
    for word in processed_reviews[i]:
      try:
        temp = (1 if model.wv.similarity(word, word_pair[0])>model.wv.similarity(word, word_pair[1]) else -1)
        sum+= temp
      except:
        pass
      
    if (sum>0 and labels[i]=='positive') or (sum<=0 and labels[i]=='negative'):
      correct_count+=1
    # print(labels[i],avg_g,avg_b,avg_g>avg_b)
  print(word_pair,correct_count/len(processed_reviews))

['good', 'bad'] 0.6237466274715097
['best', 'worst'] 0.5518664680062819
['positive', 'negative'] 0.5527221841903918
['awesome', 'terrible'] 0.6340756251761769


In [ ]:
for j in range(50,150,5):
  result = []
  correct_count = 0
  for i in range(len(processed_reviews)):
    sum_g = 0
    sum_b = 0
    n=0
    for word in processed_reviews[i]:
      try:
        sum_g += model.wv.similarity(word, 'good')
        sum_b += model.wv.similarity(word, 'bad')
      except:
        n-=1
    if sum_g>0:
      avg_g = sum_g/(len(processed_reviews[i])+n)
    else:
      avg_g =0
    if sum_b>0:
      avg_b = sum_b/(len(processed_reviews[i])+n)
    else:
      avg_b = 0
    if (avg_b==0 and labels[i]=='positive'):
      correct_count+=1
    elif avg_b!=0:
      if (avg_g/avg_b>j/100 and labels[i]=='positive') or (avg_g/avg_b<=j/100 and labels[i]=='negative'):
        correct_count+=1

  print('Threshold: ',j/100 ,' - Accuracy:',correct_count/len(processed_reviews))

Threshold:  0.5  - Accuracy: 0.523335883703137
Threshold:  0.55  - Accuracy: 0.531017194861676
Threshold:  0.6  - Accuracy: 0.5400273829178915
Threshold:  0.65  - Accuracy: 0.5514235090403898
Threshold:  0.7  - Accuracy: 0.5661720291547537
Threshold:  0.75  - Accuracy: 0.5832561510892764
Threshold:  0.8  - Accuracy: 0.6010852494664358
Threshold:  0.85  - Accuracy: 0.6196190552893327
Threshold:  0.9  - Accuracy: 0.6383944750936255
Threshold:  0.95  - Accuracy: 0.6552973865421012
Threshold:  1.0  - Accuracy: 0.6698445616719687
Threshold:  1.05  - Accuracy: 0.682146740224701
Threshold:  1.1  - Accuracy: 0.6911468610316917
Threshold:  1.15  - Accuracy: 0.6987375669472073
Threshold:  1.2  - Accuracy: 0.7019490194499255
Threshold:  1.25  - Accuracy: 0.7041336125317118
Threshold:  1.3  - Accuracy: 0.7026235251479885
Threshold:  1.35  - Accuracy: 0.6992912656545726
Threshold:  1.4  - Accuracy: 0.6932710506181291
Threshold:  1.45  - Accuracy: 0.687401844320058
